In [43]:
import pandas as pd

In [44]:
df = pd.read_csv("spotify_millsongdata.csv")

In [45]:
df.head(5)

,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \r\nA..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \r\nTouch me gen..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \r\nWhy I had...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [46]:
df.tail(5)

,artist,song,link,text
57645,Ziggy Marley,Good Old Days,/z/ziggy+marley/good+old+days_10198588.html,Irie days come on play \r\nLet the angels fly...
57646,Ziggy Marley,Hand To Mouth,/z/ziggy+marley/hand+to+mouth_20531167.html,Power to the workers \r\nMore power \r\nPowe...
57647,Zwan,Come With Me,/z/zwan/come+with+me_20148981.html,all you need \r\nis something i'll believe \...
57648,Zwan,Desire,/z/zwan/desire_20148986.html,northern star \r\nam i frightened \r\nwhere ...
57649,Zwan,Heartsong,/z/zwan/heartsong_20148991.html,come in \r\nmake yourself at home \r\ni'm a ...


In [47]:
df.shape

(57650, 4)

In [48]:
df.isnull().sum()

artist    0
song      0
link      0
text      0
dtype: int64

In [49]:
df =df.sample(5000).drop('link', axis=1).reset_index(drop=True)

In [50]:
df.head(10)

,artist,song,text
0,Dean Martin,Deep Purple,When the deep purple falls \r\nOver sleepy ga...
1,Il Divo,Adeste Fideles (O Come All Ye Faithful),"O, come all ye faithful \r\nJoyful and triump..."
2,Utopia,Too Much Water,"Too much, it's too much, too much \r\nGot too..."
3,Keith Green,Altar Call,You know I can't explain to you \r\nReally ho...
4,Louis Armstrong,That's My Desire,To spend one night with you \r\nIn our old re...
5,Leann Rimes,This Love,In every life there are detours on dirt gravel...
6,George Jones,Billy B. Bad,Down in Texas near the DFW among the shrubs an...
7,Morrissey,At Last I Am Born,At last I am born \r\nHistorians note \r\nI ...
8,Emmylou Harris,Lost Unto This World,I was once some mother's darlin' \r\nSome dad...
9,Rihanna,Winning Women,Nicole. \r\nRi-Rihanna. \r\nNi-Ni-Nicole. \...


In [51]:
df['text'][0]

"When the deep purple falls  \r\nOver sleepy garden walls  \r\nAnd the stars begin to flicker in the sky  \r\nThrough the mist of a memory  \r\nYou wander back to me  \r\nBreathing my name with a sigh  \r\nIn the still of the night  \r\nOnce again I'll hold you tight  \r\nThough you're gone your love lives on  \r\nWhen moonlight beams  \r\nAnd as long as my heart will beat  \r\nLove we'll always meet  \r\nHere in my deep purple dreams  \r\nAnd as long as my heart will beat  \r\nLove we'll always meet  \r\nHere in my deep purple dreams\r\n\r\n"

In [52]:
# df = df.sample(5000)

In [53]:
df.shape

(5000, 3)

Text Cleaning/ Text Preprocessing

In [54]:
df['text'] = df['text'].str.lower().replace(r'^\w\s', ' ').replace(r'\n', ' ', regex = True)

In [55]:
import nltk
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def tokenization(txt):
    tokens = nltk.word_tokenize(txt)
    stemming = [stemmer.stem(w) for w in tokens]
    return " ".join(stemming)


In [56]:
# df['text'] = df['text'].apply(lambda x: tokenization(x))

In [57]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [58]:
tfidvector = TfidfVectorizer(analyzer='word',stop_words='english')
matrix = tfidvector.fit_transform(df['text'])
similarity = cosine_similarity(matrix)

In [59]:
similarity[0]

array([1.        , 0.        , 0.        , ..., 0.02796377, 0.02215044,
       0.01132952])

In [60]:
df[df['song'] == 'Crying Over You']

,artist,song,text
3843,ABBA,Crying Over You,i'm waitin' for you baby \r i'm sitting all a...


In [61]:
def recommendation(song_df):
    idx = df[df['song'] == song_df].index[0]
    distances = sorted(list(enumerate(similarity[idx])),reverse=True,key=lambda x:x[1])
    
    songs = []
    for m_id in distances[1:21]:
        songs.append(df.iloc[m_id[0]].song)
        
    return songs

In [62]:
print(df)

                  artist                                     song   
0            Dean Martin                              Deep Purple  \
1                Il Divo  Adeste Fideles (O Come All Ye Faithful)   
2                 Utopia                           Too Much Water   
3            Keith Green                               Altar Call   
4        Louis Armstrong                         That's My Desire   
...                  ...                                      ...   
4995       Fifth Harmony                                     Boss   
4996       Reba Mcentire                                On My Own   
4997  Kris Kristofferson                            Risky Bizness   
4998             Madonna                           Graffiti Heart   
4999           Chris Rea                             Let It Loose   

                                                   text  
0     when the deep purple falls  \r over sleepy gar...  
1     o, come all ye faithful  \r joyful and triumph...

In [63]:
recommendation('Crying Over You')

['Moment Of Forever',
 'Feeling Blue',
 "Don't Let The Sun Catch You Cryin'",
 'Please Send Me Someone To Love',
 'I Want You To Want Me',
 'Blue Christmas',
 'The Storm Is Over Now',
 'Do Not Pass Me By',
 '2:10 Train',
 'Drift Away',
 'People Need The Lord',
 "I'm Looking For Blue Eyes",
 'Jesus Is Lord Of All',
 'Blues For Sister Someone',
 'Trading My Sorrows (Yes Lord)',
 'Use Somebody',
 'Sinnerman',
 "I Know (You Don't Love Me)",
 "Don't Wanna Fall Away From You",
 'Viva La Vida']

In [64]:
import pickle
pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(df,open('df.pkl','wb'))